In [1]:
# processing
import operator
from operator import methodcaller
import csv
import re
import numpy as np
import pandas as pd
from pprint import pprint
import string
import math
import itertools
import sqlite3
import copy

# gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import HdpModel
from gensim.models import TfidfModel

# nltk
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer

# plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 


C:\Users\pjz1\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [12]:
# path to blacklisted tokens
blacklist = [t.strip() for t in next(csv.reader(open("tools\\blacklist.csv", 'r')))]

# levels of R&R terms considered
levels = [1, 2, 3]

# format [term, orig, sentence, docID]
inPath = "raw.csv"
inFile = open(inPath, 'r')
inReader = csv.reader(inFile)

In [13]:

docTokens = dict()


next(inReader)
for inRow in inReader:
    term = inRow[0]
    sentence = inRow[2]
    docID = inRow[3]
    
    token = "_".join([t for t in term.split(":") if re.match(r'[^\W\d]*$', t) and not t in blacklist])
    
    level = token.count("_")
    
    if level in levels and not token in blacklist and len(token) > 0:
        if docID in docTokens:
            docTokens[docID] += [token]
        else:
            docTokens[docID] = [token]

docIDs = list(docTokens.keys())
data = list(docTokens.values())

In [14]:
dictionary = corpora.Dictionary(texts)
print(len(dictionary))

dictionary.filter_extremes(no_below=2, no_above=1, keep_n=15000)
print(len(dictionary))

corpus = [dictionary.doc2bow(text) for text in texts]


tfidf = TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

16391
1705


In [15]:
model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=6, id2word=dictionary, passes=10, workers =4, iterations=500)

coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
print(coherencemodel.get_coherence())


0.4208800786237962
